In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
!%%cache
!pip install rpy2

/bin/bash: line 0: fg: no job control
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!%%cache
from google.colab import drive

drive.mount('/content/drive')

/bin/bash: line 0: fg: no job control
Mounted at /content/drive


##**Data loading**

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

x_train = pd.read_csv('drive/MyDrive/SP/x_train.csv') 
y_train = pd.read_csv('drive/MyDrive/SP/y_train.csv') 
#x_test = pd.read_csv('drive/MyDrive/SP/x_test.csv')
#y_test = pd.read_csv('drive/MyDrive/SP/y_test.csv')

df_train = pd.concat([x_train, y_train], axis=1, join='inner')
#df_test = pd.concat([x_test, y_test], axis=1, join = 'inner')


#df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

#saving validation and test split
#df_test.to_csv('drive/MyDrive/SP/NLP_test_case_validation.csv', index = False, encoding='utf-8')
#df_val.to_csv('drive/MyDrive/SP/NLP_validation.csv', index = False, encoding='utf-8')

df_test = pd.read_csv('drive/MyDrive/SP/NLP_test_case_validation.csv')
df_val = pd.read_csv('drive/MyDrive/SP/NLP_validation.csv')

In [5]:
df_train.review

0        having great grandfather captured sent changi ...
1        young ladys bonnie polay shes attractive appar...
2        animated film bored 6 grown grew area wild hor...
3        long ago 1960 forgotten series wished reappear...
4        great watch movie completely surprises me come...
                               ...                        
42495    quick simple love movie mentioned also south d...
42496    film opened poor showings weeks meena kumari d...
42497    near guinea pig flicks inspired flowers flesh ...
42498    1st watched 6212001  2 10diremmanuel itier pre...
42499    despite previous reviewers screed paced intere...
Name: review, Length: 42500, dtype: object

##**BERT**

In [6]:
!%%cache
!pip install -q -U watermark

/bin/bash: line 0: fg: no job control


In [7]:
!%%cache
!pip install -qq transformers

/bin/bash: line 0: fg: no job control
     |████████████████████████████████| 4.4 MB 6.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.2 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 


In [8]:
!%%cache
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

/bin/bash: line 0: fg: no job control
Python implementation: CPython
Python version       : 3.7.13
IPython version      : 5.5.0

numpy       : 1.21.6
pandas      : 1.3.5
torch       : 1.12.0+cu113
transformers: 4.20.1



In [9]:
#@title Setup & Config
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
PRE_TRAINED_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'

In [11]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Trying tokenizer and encoding

In [115]:
sample_txt = 'Introduction to Natural Language Processing '

In [116]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: Introduction to Natural Language Processing 
   Tokens: ['introduction', 'to', 'natural', 'language', 'processing']
Token IDs: [4955, 2000, 3019, 2653, 6364]


In [117]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


dict_keys(['input_ids', 'attention_mask'])

Finding max length

In [15]:
'''
token_lens = []

for txt in df_train.review:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))

element_count = len([item for item in token_lens if item > 256])
element_count
'''

'\ntoken_lens = []\n\nfor txt in df_train.review:\n  tokens = tokenizer.encode(txt, max_length=512)\n  token_lens.append(len(tokens))\n\nelement_count = len([item for item in token_lens if item > 256])\nelement_count\n'

In [16]:
'''
sns.distplot(token_lens)
plt.xlim([0, 512]);
plt.xlabel('Token count');
'''

"\nsns.distplot(token_lens)\nplt.xlim([0, 512]);\nplt.xlabel('Token count');\n"

In [17]:
MAX_LEN = 512

**Data preprocessing**

In [18]:
class MovieReviewsDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'review': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [19]:
df_train.shape, df_val.shape, df_test.shape

((42500, 2), (3750, 2), (3750, 2))

In [20]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = MovieReviewsDataset(
    reviews=df.review.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [21]:
BATCH_SIZE = 32

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [22]:
data = next(iter(train_data_loader))
data.keys()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/py

dict_keys(['review', 'input_ids', 'attention_mask', 'targets'])

In [23]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

torch.Size([32, 512])
torch.Size([32, 512])
torch.Size([32])


**Classifier**

In [24]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME) #PRE_TRAINED_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

In [25]:
o = bert_model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask']
)

In [26]:
last_hidden_state = o.last_hidden_state

In [27]:
last_hidden_state.shape

torch.Size([1, 32, 384])

In [28]:
pooler_output = o.pooler_output

In [29]:
pooler_output.shape

torch.Size([1, 384])

In [30]:
bert_model.config.hidden_size

384

In [31]:
class MovieReviewsClassifier(nn.Module):

  def __init__(self, n_classes):
    super(MovieReviewsClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    ob = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(ob.pooler_output)
    return self.out(output)

In [32]:
class_names = ['positive', 'negative'] 

In [33]:
model = MovieReviewsClassifier(len(class_names))
model = model.to(device)

In [34]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([32, 512])
torch.Size([32, 512])


In [35]:
F.softmax(model(input_ids, attention_mask), dim=1)

tensor([[0.5097, 0.4903],
        [0.5177, 0.4823],
        [0.5247, 0.4753],
        [0.4966, 0.5034],
        [0.5072, 0.4928],
        [0.5216, 0.4784],
        [0.5095, 0.4905],
        [0.5268, 0.4732],
        [0.5110, 0.4890],
        [0.5179, 0.4821],
        [0.5337, 0.4663],
        [0.5139, 0.4861],
        [0.5056, 0.4944],
        [0.5220, 0.4780],
        [0.4915, 0.5085],
        [0.4977, 0.5023],
        [0.5068, 0.4932],
        [0.5089, 0.4911],
        [0.4913, 0.5087],
        [0.5146, 0.4854],
        [0.5130, 0.4870],
        [0.4917, 0.5083],
        [0.5032, 0.4968],
        [0.5139, 0.4861],
        [0.4893, 0.5107],
        [0.5049, 0.4951],
        [0.5047, 0.4953],
        [0.5100, 0.4900],
        [0.5138, 0.4862],
        [0.5215, 0.4785],
        [0.5154, 0.4846],
        [0.5186, 0.4814]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

**Training**

In [36]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [37]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [38]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [39]:
%%time

#remove when training
%%script echo skipping 
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

skipping
CPU times: user 1.74 ms, sys: 34.6 ms, total: 36.3 ms
Wall time: 69.9 ms


In [40]:
import pickle
tmp = history['train_acc']

f = open("drive/MyDrive/SP/NLP_history.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(history,f)

f.close()

NameError: ignored

In [ ]:
#file_to_read = open("drive/MyDrive/SP/NLP_history.pkl", "rb")

#history = pickle.load(file_to_read)
history_train_acc = torch.Tensor.cpu(torch.tensor(history['train_acc']))
history_val_acc = torch.Tensor.cpu(torch.tensor(history['val_acc']))

In [ ]:
plt.plot(history_train_acc, label='train accuracy')
plt.plot(history_train_acc, label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

In [41]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  sar_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["review"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      sar_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return sar_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

## **Saving predictions and the model**

In [65]:
from os import path

def json_metrics(file_name, prediction_model, metrics, df):
    dictionary = {'Model': prediction_model,
                  'Metrics': metrics,
                  'Data': df.to_dict('records')}

    if path.isfile(file_name): #file exist
        with open(file_name) as fp:
            listObj = json.load(fp)

        listObj.append(dictionary)

        with open(file_name, 'w') as json_file:
            json.dump(listObj, json_file, indent=4)
    else:
        with open(file_name, 'w') as json_file:
            json.dump([dictionary], json_file, indent=4)

In [ ]:
from sklearn import metrics as m
import json
def metrics(y_test, y_pred, target_names):
    tn, fp, fn, tp = m.confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
    dict_confusion = {'True negative' : int(tn),
          'False positive' : int(fp),
          'False negative' : int(fn),
          'True positive' : int(tp),
          }
    dict_report = m.classification_report(y_test, y_pred, target_names=target_names, output_dict=True)
    return {**dict_confusion , **dict_report}

In [ ]:
metrics = metrics(y_test, y_pred, ['1','0'])

In [ ]:
y_test_ids = np.arange(y_test.shape[0])

In [ ]:
#dataframe
df= pd.DataFrame({'id': y_test_ids, 'label': y_test, 'prediction': y_pred})

**remove # when writing new results**

In [ ]:
#json_metrics("drive/MyDrive/SP/NLP_Bert.json", "sentence-transformers/all-MiniLM-L6-v2", metrics, df )

In [ ]:
!ls

**remove # when saving new model**

In [ ]:

#torch.save(model, 'drive/MyDrive/SP/NLP_Bert_Model')

## **Neutral Dataset**

In [42]:
import pandas as pd

saved_model = torch.load('drive/MyDrive/SP/NLP_Bert_Model')

df_neutral = pd.read_csv('drive/MyDrive/SP/neutral_dataset.csv')
df_neutral.rename(columns = {'Phrase':'review', 'Sentiment':'sentiment'}, inplace = True)
test_data_loader = create_data_loader(df_neutral, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [43]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  saved_model,
  test_data_loader
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/py

In [103]:
def metrics_for_neutral(y_test, y_pred, target_names):
  FP = TP = TN = FN = NP = NN = 0

  for i in range(len(y_pred)):
    prediction = y_pred.numpy()[i]
    true = y_test.numpy()[i]

    if prediction == 1 and (true == 1 or true == 0):
      FP += 1
    elif prediction == 1 and (true == 3 or true == 4):
      TP += 1
    elif prediction == 0 and (true == 1 or true == 0) :
      TN += 1
    elif prediction == 0 and (true == 3 or true == 4):
      FN += 1
    elif prediction == 1 and true == 2:
      NP += 1
    elif prediction == 0 and true == 2:
      NN += 1
      
  dict_confusion = {'True negative' : int(TN),
        'False positive' : int(FP),
        'False negative' : int(FN),
        'True positive' : int(TP),
        'Neutral positive' : int(NP),
        'Neutral negative' : int(NN),
        }
  return {**dict_confusion}

In [104]:
metrics_for_neutral = metrics_for_neutral(y_test, y_pred, ['1','0'])

In [106]:
metrics_for_neutral

{'False negative': 730,
 'False positive': 828,
 'Neutral negative': 838,
 'Neutral positive': 817,
 'True negative': 2444,
 'True positive': 2872}

In [107]:
y_test_ids_neutral = df_neutral['SentenceId'].to_numpy()

In [108]:
df_neutral_json= pd.DataFrame({'id': y_test_ids_neutral, 'label': y_test, 'prediction': y_pred})

In [109]:
import json
json_metrics("drive/MyDrive/SP/NLP_Bert_Neutral.json", "sentence-transformers/all-MiniLM-L6-v2", metrics_for_neutral, df_neutral_json )